## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd
import gzip

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [6]:
MIN_WORD_COUNT = 300
NUM_CORES = 16

In [7]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [8]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"
GZIP_EXTENSION = ".gz"

In [9]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join(root_location, "parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks", "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [10]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 22.1 s, sys: 1.24 s, total: 23.3 s
Wall time: 23.3 s


In [11]:
len(training_docs_list)

1286325

In [12]:
len(validation_docs_list)

321473

In [13]:
len(test_docs_list)

401877

# Utility functions for data loading

In [14]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [15]:
VALIDATION_MINI_BATCH_SIZE = 100000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchClass(preprocessed_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        while True:
            pool = ThreadPool(NUM_CORES)
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
            pool.close()
            pool.terminate()
        #pool.close()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [16]:
class ExtendedPVDocumentBatchGeneratorMine(object):
    def __init__(self, filename_prefix, batch_size=10000 ):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        self.filename_prefix = filename_prefix
        self.curr_lines = []
        self.curr_docids = []
        self.batch_size = batch_size
        self.curr_index = 0
        self.curr_doc_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        del self.curr_lines, self.curr_docids
        self.curr_lines, self.curr_docids = [], []
        info("Loading new batch for index: {}".format(self.curr_doc_index))
        #if self.curr_doc_index > 0: self.curr_doc_index -= 1
        true_docs_count = 0
        try:
            with open(self.filename_prefix + str(self.curr_doc_index)) as preproc_file:
                for line in preproc_file:
                    line_array = line.split(" ")
                    doc_id = line_array[0]
                    doc_tokens = line_array[1:]
                    self.curr_lines.append(doc_tokens)
                    self.curr_docids.append(doc_id)
                    if is_real_doc(doc_id):
                        true_docs_count+= 1
            self.batch_end = self.curr_doc_index + true_docs_count - 1 
            info("Finished loading new batch of {} documents".format(true_docs_count))
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_index))
            raise StopIteration()
    def __iter__(self):
        while True:
            if self.curr_doc_index > self.batch_end:
                self.load_new_batch_in_memory()
            for (doc_id, tokens) in zip(self.curr_docids, self.curr_lines):
                if self.batch_size is not None:
                    curr_batch_iter = 0
                    # divide the document to batches according to the batch size
                    while curr_batch_iter < len(tokens):
                        yield LabeledSentence(words=tokens[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id])
                        curr_batch_iter += self.batch_size
                else:
                    yield doc_id, tokens
                self.curr_index += 1
                # increment only for full docs
                if is_real_doc(doc_id):
                    self.curr_doc_index += 1
                    if self.curr_doc_index % 1000 == 0:
                        info("New Doc: {:5}, Batch End: {:5}".format(self.curr_doc_index, self.batch_end))

def is_real_doc(doc_id):
    return doc_id.find("_") == -1






class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.batch_size = batch_size
        self.cur_file = None
        self.files_loaded = start_file - offset

    def line_processing(self, line, batch_size):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        if batch_size is not None:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                self.queue.put(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + batch_size], tags=[doc_id]), block=True, timeout=None)
                curr_batch_iter += batch_size
        else:
            self.queue.put(LabeledSentence(words=line_array, tags=[doc_id]), block=True, timeout=None)
        return

    def run(self):
        while True:
            try:
                if self.cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    self.cur_file = gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + GZIP_EXTENSION)
                    self.files_loaded += self.offset
                for line in self.cur_file:
                    self.line_processing(line, self.batch_size)
                self.cur_file.close()
                self.cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000):
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                raise StopIteration()
            else:
                yield item

# Doc2vec and SVM Parameters

In [17]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model

In [18]:
placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}'.format(DOC2VEC_SIZE, 
                                                                DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE))
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
placeholder_model_name

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_{}'

In [19]:
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE , window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT, 
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

## First: construct (or load) the vocabulary
Only needed to be run if you dont already haave at least one epoch computed, otherwise, just set the start_from (below) to the epoch you want to restart from

In [20]:
%%time
# training_docs_iterator = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, batch_size=10000)
training_docs_iterator = BatchClass(training_preprocessed_files_prefix, batch_size=10000)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-06 05:57:37,601 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/vocab_model/model
2017-04-06 05:57:37,602 : INFO : Loading new file for index: 0
2017-04-06 06:00:49,254 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/vocab_model/model.docvecs.* with mmap=None
2017-04-06 06:00:49,255 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/vocab_model/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-06 06:01:07,168 : INFO : loading doctag_syn0_lockf from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/vocab_model/model.docvecs.doctag_syn0_lockf.npy with mmap=None
2017-04-06 06:01:07,265 : INFO : loading syn1neg from /mnt/virtual-machines/data/parameter_search_doc2v

CPU times: user 12min 50s, sys: 1min 9s, total: 14min
Wall time: 14min


In [21]:
# vocab_counts = {k:doc2vec_model.vocab[k].count for k in doc2vec_model.vocab.keys()}
# dd = sorted(vocab_counts, key=vocab_counts.get)

## Actual Training, validation and Metrics Loop

In [20]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [21]:
doc2vec_model.workers

16

In [20]:
doc2vec_model.workers = NUM_CORES

In [21]:
BatchClass = BatchWrapper
# BatchClass = ExtendedPVDocumentBatchGeneratorMine

In [21]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 8
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
#         training_docs_iterator = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, batch_size=10000)
        training_docs_iterator = BatchClass(training_preprocessed_files_prefix, batch_size=10000)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch > 7:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix)

2017-04-10 12:28:09,227 : INFO : ****************** Epoch 8 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8 *******************
2017-04-10 12:28:09,228 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model
2017-04-10 12:31:51,008 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.* with mmap=None
2017-04-10 12:31:51,009 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.do

CPU times: user 2min 56s, sys: 55.9 s, total: 3min 52s
Wall time: 4min 47s


## Inference Only (if needed)

In [22]:
NUM_CORES = 16

In [23]:
epoch = 8
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

2017-04-10 13:20:24,127 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model
2017-04-10 13:23:42,553 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.* with mmap=None
2017-04-10 13:23:42,554 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-10 13:24:29,762 : INFO : loading doctag_syn0_lockf from /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_ful

In [24]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix)

2017-04-10 13:32:51,264 : INFO : ===== Getting vectors with inference
2017-04-10 13:32:52,408 : INFO : Loading new file for index: 0
2017-04-10 13:33:43,983 : INFO : Finished: 100000 tags
2017-04-10 13:35:19,997 : INFO : Finished: 200000 tags
2017-04-10 13:36:12,646 : INFO : Finished: 300000 tags
2017-04-10 13:36:23,178 : INFO : Loading new file for index: 10000
2017-04-10 13:37:05,034 : INFO : Finished: 400000 tags
2017-04-10 13:37:57,255 : INFO : Finished: 500000 tags
2017-04-10 13:38:49,978 : INFO : Finished: 600000 tags
2017-04-10 13:39:09,159 : INFO : Loading new file for index: 20000
2017-04-10 13:39:42,036 : INFO : Finished: 700000 tags
2017-04-10 13:40:34,778 : INFO : Finished: 800000 tags
2017-04-10 13:41:26,601 : INFO : Finished: 900000 tags
2017-04-10 13:42:18,368 : INFO : Finished: 1000000 tags
2017-04-10 13:42:23,685 : INFO : Loading new file for index: 30000
2017-04-10 13:43:10,199 : INFO : Finished: 1100000 tags
2017-04-10 13:44:02,725 : INFO : Finished: 1200000 tags
201

KeyboardInterrupt: 